In [1]:
## Remove below from your notebook, example of setting variables in key vault

In [ ]:
registered_env_name = "Prediction_env"
experiment_folder = 'devOps_train_pipeline'
cluster_name = "wipfli-cluster"
conda_yml_file = './Environment/Prediction_env.yml'
model_name = 'wipfli'

In [16]:
# Import required packages
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
import logging
from azureml.core.model import Model
from azureml.exceptions import WebserviceException

In [17]:
import azureml.core
import os, shutil
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset
from azureml.core.authentication import ServicePrincipalAuthentication

def get_environment_variables():
    global envs
    global run_by_notebook 
    run_by_notebook = False
    environment_variables = ['tenantId', 'servicePrincipalId', 'servicePrincipalPassword', 'wsName', 
                         'subscriptionId', 'resourceGroup']
    envs = {}
    for x in environment_variables:
        if os.environ.get(x) == None:
            #get the values from keyvault
            run_by_notebook = True
            print('retrieve from key vault, value is None: ' + x)
            ws = Workspace.from_config()
            keyvault = ws.get_default_keyvault()
            kv_results = keyvault.get_secrets(environment_variables)
            envs = kv_results
            for x in envs:
                os.environ.setdefault(x, envs[x])
            exit
        else:
            envs[x] = os.environ.get(x)
    return run_by_notebook



get_environment_variables()
sp = ServicePrincipalAuthentication(tenant_id=envs['tenantId'], # tenantID
                                    service_principal_id=envs['servicePrincipalId'], # clientId
                                    service_principal_password=envs['servicePrincipalPassword']) # clientSecret
ws = Workspace.get(name=envs['wsName'],
                       auth=sp,
                       subscription_id=envs['subscriptionId'],
                       resource_group=envs['resourceGroup'])
ws.get_details()

print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
print('Run By Notebook:' + str(run_by_notebook))
# Get the default datastore
default_ds = ws.get_default_datastore()

Ready to use Azure ML 1.47.0 to work with xmm-devops-final-dev
Run By Notebook:False


## Select Compute

In [18]:
#Select AML Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [ ]:
try:
    initial_model = Model(ws, model_name)
    inital_model_version = initial_model.version
except WebserviceException :
    inital_model_version = 0
print('inital_model_version = ' + str(inital_model_version))

## Create Run Config

In [20]:
import os
import shutil
# Create a folder for the pipeline step files
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

run_path = './run_outputs'
try:
    shutil.rmtree(run_path)
except:
    print('continue run_outputs directory does not exits')

devOps_train_pipeline
continue run_outputs directory does not exits


In [21]:
env = Environment.from_conda_specification("Prediction-env", conda_yml_file)
env.register(workspace=ws)

{
    "assetId": "azureml://locations/eastus/workspaces/d804027a-0ec6-475a-9b96-b16d84c39b43/environments/Prediction-env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Prediction-env",
    "p

In [22]:
registered_env_name

'Prediction_env'

In [24]:
# Create a Python environment for the experiment (from a .yml file)

try:
    env = Environment.from_conda_specification("Prediction_env", conda_yml_file)
    env.register(workspace=ws)
    registered_env = Environment.get(ws, registered_env_name)
    pipeline_run_config = RunConfiguration()
    
    # Use the compute you created above. 
    pipeline_run_config.target = pipeline_cluster

    # Assign the environment to the run configuration
    pipeline_run_config.environment = registered_env
    print ("Run configuration created.")
except Exception as e: 
    print(e)

Run configuration created.


In [25]:
training_data  = OutputFileDatasetConfig(name='training_data', destination=(default_ds, prefix + '_training_data/{run-id}')).read_delimited_files().register_on_complete(name= prefix + '_training_data')
testing_data   = OutputFileDatasetConfig(name='testing_data', destination=(default_ds, prefix + '_testing_data/{run-id}')).read_delimited_files().register_on_complete(name= prefix + '_testing_data')

model_file     = PipelineData(name='model_file', datastore=default_ds)
model_name         = PipelineParameter("model_name", default_value=model_name)
model_desc         = PipelineParameter("model_desc", default_value=model_name + ' description')
raw_file_location  = PipelineParameter(name="raw_file_location", default_value='Wipfli/raw_data.csv')

In [121]:
%%writefile $experiment_folder/training.py

import os
import sys
import argparse
import joblib
import pandas as pd
import numpy as np
import shutil

from azureml.core import Run, Dataset, Workspace, Experiment

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics
from sklearn.metrics import roc_auc_score,roc_curve, log_loss

# Calculate model performance metrics
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

def getRuntimeArgs():
    parser = argparse.ArgumentParser()
    
    parser = argparse.ArgumentParser()
    parser.add_argument("--raw_file_location", type=str)
    parser.add_argument('--training_data', dest='training_data', required=True)
    parser.add_argument('--testing_data', dest='testing_data', required=True)
    parser.add_argument('--model_file', dest='model_file', required=True)
    parser.add_argument('--model_name', dest='model_name', required=True)
    args = parser.parse_args()
    return args

def buildpreprocessorpipeline(X_raw):
    categorical_features = X_raw.select_dtypes(include=['object']).columns
    numeric_features = X_raw.select_dtypes(include=['float','int64']).columns

    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="missing")),
                                              ('onehotencoder', OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore'))])
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_features),
            ('categorical', categorical_transformer, categorical_features)
        ], remainder="drop")
    
    return preprocessor

def model_train(LABEL, df, run, training_data, testing_data):  
#     y_raw = df[LABEL]
#     X_raw = df.drop([LABEL], axis=1)
    
     # Train test split
    train, test = train_test_split(df, test_size=0.2, random_state=0)
    
    train1 = train.dropna()
    test1 = test.dropna()
    
    X_train = train1
    y_train = train1[LABEL]
    
    X_test = test1
    y_test = test1[LABEL]

    
    #save train and test datasets
    os.makedirs(training_data, exist_ok=True)
    os.makedirs(testing_data, exist_ok=True)

    train1.to_csv(os.path.join(training_data, 'training_data.csv'), index=False )
    test1.to_csv(os.path.join(testing_data, 'testing_data.csv'), index=False)

    
    lg = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    preprocessor = buildpreprocessorpipeline(X_train)
    
    #estimator instance
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', lg)])

    clf.fit(X_train, y_train)

    
    
    # calculate AUC
    y_scores = clf.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))
    run.log('AUC', np.float(auc))

    
    # calculate test accuracy
    y_hat = clf.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)
    run.log('Accuracy', np.float(acc))


    precisions, recall, f1_score, _ = metrics.precision_recall_fscore_support(y_test, y_hat)
    
    run.log('precision', np.float(precisions[1]))
    run.log('recall', np.float(recall[1]))
    y_pred_prob = clf.predict_proba(X_test)[::,1]
    run.log('Log-Loss', np.float(log_loss(y_test, y_pred_prob)))
    
    run.parent.log('precision', np.float(precisions[1]))
    run.parent.log('recall', np.float(recall[1]))
    run.parent.log('Log-Loss', np.float(log_loss(y_test, y_pred_prob)))


    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_hat)
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    run.log_image(name = "ROC", plot = fig)
    plt.show()

    # plot confusion matrix
    # Generate confusion matrix
    cmatrix = confusion_matrix(y_test, y_hat)
    cmatrix_json = {
        "schema_type": "confusion_matrix",
           "schema_version": "v1",
           "data": {
               "class_labels": ["0", "1"],
               "matrix": [
                   [int(x) for x in cmatrix[0]],
                   [int(x) for x in cmatrix[1]]
               ]
           }
    }
    
    run.log_confusion_matrix('ConfusionMatrix_Test', cmatrix_json)

    return clf
    # Save the trained model
    
    
def main():
    # Create an Azure ML experiment in your workspace
    args = getRuntimeArgs()
    
    raw_file_location = args.raw_file_location
    training_data = args.training_data
    testing_data = args.testing_data
    model_file = args.model_file
    model_name = args.model_name
    
    
    run = Run.get_context()

    dataset_dir = './dataset/'
    os.makedirs(dataset_dir, exist_ok=True)
    ws = run.experiment.workspace
    ds = ws.get_default_datastore()
    
    print(ws)
    

    print("Loading Data...")
    dataset = Dataset.Tabular.from_delimited_files(path = [(ds, raw_file_location)])
    # Load a TabularDataset & save into pandas DataFrame
    df = dataset.to_pandas_dataframe()
    
    print(df.head(5))
 
    model = model_train('Survived', df, run, training_data, testing_data)
    
    # Save the trained model
#     model_file = 'titanic_model.pkl'
#     joblib.dump(value=model, filename='./outputs/' + model_file)
#     os.makedirs(model_file_output, exist_ok=True)
#     shutil.copyfile('./outputs/' + model_file, os.path.join(model_file_output, 'titanic_model.pkl'))

    os.makedirs('./outputs', exist_ok=True)

    model_file_name = model_name  + '.pkl'
    file_name = './outputs/' +model_file_name

    joblib.dump(value=model, filename=file_name)

    os.makedirs(model_file, exist_ok=True)

    shutil.copyfile(file_name, os.path.join(model_file, model_file_name))
    

if __name__ == "__main__":
    main()

Overwriting devOps_train_pipeline/training.py


In [122]:

train_model_step = PythonScriptStep(
    name='Get Data and Create Model',
    script_name='training.py',
    arguments =['--raw_file_location', raw_file_location,
                '--training_data', testing_data,
                '--testing_data', testing_data,
                '--model_file', model_file,
                '--model_name', model_name
               ],
    inputs=[],
    outputs=[model_file, training_data, testing_data],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [123]:
%%writefile $experiment_folder/eval.py

from azureml.core import Run, Workspace, Datastore, Dataset
from azureml.core.model import Model
from azureml.data.datapath import DataPath

import joblib
import os
import argparse
import shutil
import pandas as pd
import shutil


from azureml.core.model import InferenceConfig
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.webservice import Webservice, AksWebservice


parser = argparse.ArgumentParser("Evaluate model and register if more performant")

parser.add_argument('--model_name', dest='model_name', required=True)
parser.add_argument('--model_file', dest = 'model_file',  required=True)
parser.add_argument('--model_desc', dest = 'model_desc',  required=True)
parser.add_argument('--deploy_file', dest='deploy_file', required=True)


args, _ = parser.parse_known_args()
model_name = args.model_name
model_file = args.model_file
model_desc = args.model_desc

deploy_file = args.deploy_file

#Get current run
run = Run.get_context()

#Get associated AML workspace
ws = run.experiment.workspace

#Get default datastore
ds = ws.get_default_datastore()

#Get metrics associated with current parent run
metrics = run.get_metrics()

print('current run metrics')
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')


print('parent run metrics')
#Get metrics associated with current parent run
metrics = run.parent.get_metrics()

for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')

current_model_precision = float(metrics['precision'])
current_model_recall = float(metrics['recall'])
current_model_logloss = float(metrics['Log-Loss'])
# Get current model from workspace

model_description = model_desc
model_list = Model.list(ws, name=model_name, latest=True)
first_registration = len(model_list)==0

updated_tags = {'precision': current_model_precision, 'recall': current_model_recall}


print('updated tags')
print(updated_tags)


#upload model to the outputs directory
relative_model_path = 'outputs'
run.upload_folder(name=relative_model_path, path=model_file)

model_file_name = model_name  + '.pkl'


#If no model exists register the current model
if first_registration:
    print('First model registration.')
    model_reg = run.register_model(model_path='outputs/' + model_file_name, model_name=model_name,
                   tags=updated_tags,
                   properties=updated_tags)
else:
    #If a model has been registered previously, check to see if current model 
    #performs better. If so, register it.
    print(dir(model_list[0]))
    if float(model_list[0].tags['precision']) < current_model_precision:
        print('New model performs better than existing model. Register it.')

        model_reg = run.register_model(model_path='outputs/' + model_file_name, model_name=model_name,
                   tags=updated_tags,
                   properties=updated_tags)
        
        # Output accuracy to file
        with open(deploy_file, 'w+') as f:
            f.write(('deploy_model'))
    
    else:
        print('New model does not perform better than existing model. Cancel run.')
        
        with open(deploy_file, 'w+') as f:
            f.write(('do not deploy model'))
            
        run.complete()

Overwriting devOps_train_pipeline/eval.py


In [124]:

deploy_file = PipelineData(name='deploy_file', datastore=default_ds)

evaluate_and_register_step = PythonScriptStep(
    name='Evaluate and Register Model',
    script_name='eval.py',
    arguments=[
        '--model_name', model_name,
        '--model_file', model_file,
        '--model_desc', model_desc,
        '--deploy_file', deploy_file,       
    ],
    inputs=[model_file.as_input('model_file'),
            training_data.as_input(name='training_data'),
            testing_data.as_input(name='testing_data')
           ],
    outputs=[ deploy_file],
    compute_target=pipeline_cluster,
    source_directory='./' + experiment_folder,
    allow_reuse=False,
    runconfig=pipeline_run_config
)

In [125]:
## Create Pipeline Steps
pipeline = Pipeline(workspace=ws, steps=[train_model_step, evaluate_and_register_step])
if run_by_notebook:
    experiment = Experiment(ws, 'AML_Manual_PipelineTraining')
else:
    experiment = Experiment(ws, 'AML_AutoDevOps_PipelineTraining')
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)

Created step Get Data and Create Model [32cf1268][c9337be5-9490-438a-956e-b94374c41023], (This step will run and generate new outputs)
Created step Evaluate and Register Model [93e5cf9a][17466abb-3034-49ba-b203-0cdbb9edf3aa], (This step will run and generate new outputs)
Submitted PipelineRun 1bc148c2-923d-4237-a582-05548a9538d7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1bc148c2-923d-4237-a582-05548a9538d7?wsid=/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/xmm-devops-final-dev-rg/workspaces/xmm-devops-final-dev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRunId: 1bc148c2-923d-4237-a582-05548a9538d7
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1bc148c2-923d-4237-a582-05548a9538d7?wsid=/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/xmm-devops-final-dev-rg/workspaces/xmm-devops-final-dev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 477




StepRunId: d42a06cf-dd39-453a-ac78-c950393fdba2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/d42a06cf-dd39-453a-ac78-c950393fdba2?wsid=/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/xmm-devops-final-dev-rg/workspaces/xmm-devops-final-dev&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
StepRun( Evaluate and Register Model ) Status: NotStarted
StepRun( Evaluate and Register Model ) Status: Running

StepRun(Evaluate and Register Model) Execution Summary
StepRun( Evaluate and Register Model ) Status: Finished
{'runId': 'd42a06cf-dd39-453a-ac78-c950393fdba2', 'target': 'wipfli-cluster', 'status': 'Completed', 'startTimeUtc': '2022-12-15T15:38:58.335881Z', 'endTimeUtc': '2022-12-15T15:39:14.415392Z', 'services': {}, 'properties': {'ContentSnapshotId': '415fd688-97a4-4f5f-8ece-2c60e4778d09', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.moduleid': '17466abb-3034-49ba-b203-0cdbb9edf3aa', 'azureml.moduleName': 'Evaluate and Regis



PipelineRun Execution Summary
PipelineRun Status: Finished
{'runId': '1bc148c2-923d-4237-a582-05548a9538d7', 'status': 'Completed', 'startTimeUtc': '2022-12-15T15:38:05.625252Z', 'endTimeUtc': '2022-12-15T15:39:15.460579Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{"raw_file_location":"Wipfli/raw_data.csv","model_name":"wipfli","model_desc":"wipfli description"}', 'azureml.continue_on_step_failure': 'False', 'azureml.continue_on_failed_optional_input': 'True', 'azureml.pipelineComponent': 'pipelinerun', 'azureml.pipelines.stages': '{"Initialization":null,"Execution":{"StartTime":"2022-12-15T15:38:05.9496548+00:00","EndTime":"2022-12-15T15:39:15.3717657+00:00","Status":"Finished"}}'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://xmmdevopsfinal8028030861.blob.core.windows.net/azureml/ExperimentRun/dcid.1bc148c2-923d-4237-a582-05548a9538d7/

'Finished'

In [ ]:
import json

try:
    final_model = Model(ws, model_name)
    final_model_version = final_model.version
except WebserviceException :
    final_model_version = 0
    
print('inital_model_version = ' + str(inital_model_version))
print('final_model_version= ' + str(final_model_version))

status = run.get_status()
run_details = run.get_details()

print((run_details))
print(run_details['runId'])

In [ ]:
final_model

In [ ]:
if final_model_version > 0 and (inital_model_version != final_model_version):
    deploy = 'deploy'
    model_details = {
        "name" : final_model.name,
        "version": final_model.version,
        "properties": final_model.properties,
        "nextstep": "deploy"
    }
    print(model_details)
else:
    deploy = 'no'

In [ ]:
for x in final_model.properties:
    print(x)
    print(final_model.properties[x])


print (final_model.properties)

In [ ]:
import json
import shutil
import os

outputfolder = 'run_outputs'
os.makedirs(outputfolder, exist_ok=True)

if (final_model_version != inital_model_version):
    print('new model registered')
    with open(os.path.join(outputfolder, 'deploy_details.json'), "w+") as f:
        f.write(str(model_details))
    model_name = prefix
    model_list = Model.list(ws, name=model_name, latest=True)
    model_path = model_list[0].download(exist_ok=True)
    model_file_name = prefix + '.pkl'
    shutil.copyfile(model_file_name,  os.path.join(outputfolder,model_file_name))
    
    #create model.yml file.
    with open(os.path.join(outputfolder, 'model.yml'), "w+") as f:
        f.write('$schema: https://azuremlschemas.azureedge.net/latest/model.schema.json \n')
        f.write('name: ' + model_details['name'] + '\n')
        #f.write('version: ' + str(final_model.version)  + '\n')
        f.write('path: ' + prefix + '.pkl \n')
        f.write('description: Model created from local file. \n')
        if len(final_model.properties) > 0:
            f.write('properties: ')
            f.write(json.dumps(final_model.properties))
            f.write('\n')
            f.write('tags: ')
            f.write(json.dumps(final_model.properties))
            
    
with open(os.path.join(outputfolder, 'run_details.json'), "w+") as f:
    print(run_details)
    f.write(str(run_details))

with open(os.path.join(outputfolder, "run_number.json"), "w+") as f:
    f.write(run_details['runId'])
    
with open(os.path.join(outputfolder, "deploy.txt"), "w+") as f:
    f.write(deploy)

In [ ]:
from azureml.pipeline.core import PipelineEndpoint

def published_pipeline_to_pipeline_endpoint(
    workspace,
    published_pipeline,
    pipeline_endpoint_name,
    pipeline_endpoint_description="Endpoint to Training pipeline",
):
    try:
        pipeline_endpoint = PipelineEndpoint.get(
            workspace=workspace, name=pipeline_endpoint_name
        )
        print("using existing PipelineEndpoint...")
        pipeline_endpoint.add_default(published_pipeline)
    except Exception as ex:
        print(ex)
        # create PipelineEndpoint if it doesn't exist
        print("PipelineEndpoint does not exist, creating one for you...")
        pipeline_endpoint = PipelineEndpoint.publish(
            workspace=workspace,
            name=pipeline_endpoint_name,
            pipeline=published_pipeline,
            description=pipeline_endpoint_description
        )

if deploy == 'deploy':
    print('deploy Training Pipeline')
    pipeline_endpoint_name = 'Training Pipeline'
    pipeline_endpoint_description = 'Endpoint to Training pipeline'

    published_pipeline = pipeline.publish(name=pipeline_endpoint_name,
                                         description=pipeline_endpoint_description,
                                         continue_on_step_failure=False)

    published_pipeline_to_pipeline_endpoint(
        workspace=ws,
        published_pipeline=published_pipeline,
        pipeline_endpoint_name=pipeline_endpoint_name,
        pipeline_endpoint_description=pipeline_endpoint_description
    )
else:
    print('do not publish pipeline')